<h1 align="center"><font color="red">LangGraph Swarm</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [CrewAI vs LangGraph vs AutoGen: Choosing the Right Multi-Agent AI Framework](https://www.datacamp.com/tutorial/crewai-vs-langgraph-vs-autogen)
* [LangGraph Tutorial: What Is LangGraph and How to Use It?](https://www.datacamp.com/tutorial/langgraph-tutorial)
* [agentic-rag](https://github.com/FareedKhan-dev/agentic-rag/tree/main)
* [LangChainDocs: Quickstart](https://docs.langchain.com/oss/python/langgraph/quickstart)
* [LangChainDocs: Build a custom RAG agent with LangGraph](https://docs.langchain.com/oss/python/langgraph/agentic-rag)
* [LangChainDocs: LangGraph overview](https://docs.langchain.com/oss/python/langgraph/overview)
* [Medium: Building a Tool-Driven Multi-Agent Swarm with LangGraph: When AI Agents Work Together](https://aws.plainenglish.io/building-a-tool-driven-multi-agent-swarm-with-langgraph-when-ai-agents-work-together-b699b7781bae)
* [langgraph-multi-agent-swarm](https://github.com/sampathbasa/langgraph-multi-agent-swarm/blob/main/Demo.ipynb)


Neste tutorial, vamos construir um sistema de enxame (swarm) com ``LangGraph`` mínimo com 2 agentes, cada um usando diferentes tipos de tools:

* O Agente 1 terá uma tool personalizada que definiremos.
* O Agente 2 usará a busca ``Tavily``.

# <font color="gree">``Passo 1:`` Setup</font>

Estamos garantindo que o ``LangChain``, o ``LangGraph`` e o ``wrapper do OpenAI`` estejam na versão mais recente para que todas as APIs que usarmos posteriormente estejam disponíveis. Também precisaremos da biblioteca ``laggraph-swarm`` na versão mais recente.

```bash
langchain==1.2.3,
langchain-community==0.4.1,
langchain-openai==1.1.7,
langchain-tavily==0.2.16,
langgraph==1.0.6,
langgraph-swarm==0.1.0,
rich==14.2.0,
```


# <font color="gree">``Passo 2:`` Chaves da OpenAI e Tavily</font>

Antes de executarmos o modelo, precisamos definir nossa chave ``OpenAI`` e, para que nosso sistema agente possa pesquisar na web em tempo real, usaremos o ``Tavily Search``. O agente se comunicará com o modelo ``o4-mini`` da ``OpenAI`` e realizará buscas na web. Ao armazenar sua chave em uma variável de ambiente, você permite que todas as células subsequentes se autentiquem automaticamente, sem a necessidade de codificar segredos diretamente no notebook.

Este ambiente de laboratório precisa de uma chave de API para acessar a API da OpenAI. Para acessar o Tavily, você precisa criar uma conta gratuita e gerar a chave de API manualmente.

Após gerar a chave de API do Tavily, insira-a na célula abaixo antes de executar o programa.

Você pode obter sua chave de API do Tavily em https://www.tavily.com/

In [13]:
import os

from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
azure_openai_api_key=os.environ['AZURE_OPENAI_API_KEY']
azure_openai_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
azure_apenai_api_version=os.environ['AZURE_OPENAI_API_VERSION']
azure_openai_deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
tavily_api_key=os.environ['TAVILY_API_KEY']

# <font color="gree">``Passo 3:`` Importando as bibliotecas</font>

Nesta etapa, estamos importando todas as ferramentas necessárias para construir nosso sistema multiagente.

O que estamos importando:

* ``Tool`` – Isso nos permite definir tools personalizadas. Ele encapsula uma função Python comum com metadados que informam ao agente o que a ferramenta faz, quais entradas ela espera e quando deve ser usada.

* ``AzureChatOpenAI`` – Este é o modelo de linguagem que nossos agentes usarão para pensar, raciocinar e gerar respostas.

* ``InMemorySaver`` – Uma maneira simples de armazenar o histórico de conversas na memória. Neste caso, usaremos a ``memória de curto prazo`` para que o sistema se lembre das mensagens anteriores durante a ``sessão``.

* ``create_react_agent`` – ​​Esta é uma função prática que nos permite criar rapidamente um agente com ferramentas e lógica integradas.

* ``create_handoff_tool`` – Isso permite que um agente passe a conversa para outro quando não for mais o agente adequado para lidar com a tarefa.

* ``create_swarm`` – Esta função reúne todos os agentes em um Swarm, onde eles podem trabalhar juntos e transferir tarefas conforme necessário. É isso que usaremos para construir o ambiente colaborativo.

In [14]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph_swarm import create_handoff_tool, create_swarm

Em seguida, importe e instancie a tool ``TavilySearch`` como a variável ``web_search_tool``. Essa ferramenta será passada para o agente e concederá a ele a capacidade de acessar a internet.

Defina ``max_results`` como ``1`` para limitar a quantidade de resultados retornados.

In [15]:
from langchain_tavily import TavilySearch

web_search_tool = TavilySearch(max_results=5, # Aumentar número de resultados
                              search_depth="advanced", # Usar busca mais profunda
                              include_answer=True, # Incluir resposta resumida
                              include_raw_content=False,
                              include_images=False
                              )


# <font color="gree">``Passo 4:`` Carregando o modelo</font>

Para esta demonstração, usaremos o ``GPT-4o-mini`` ou ``o4-mini``. Ele não só suporta ferramentas personalizadas, como também ferramentas pré-construídas. A ``AzureChatOpenAI`` nos fornece um objeto Python conveniente que encapsula a API do ``o4-mini``. Definir ``temperature=0.7`` torna as respostas um pouco mais criativas, mantendo-se bastante confiáveis.

In [16]:
model = AzureChatOpenAI(
    api_key=azure_openai_api_key,
    api_version=azure_apenai_api_version,
    azure_endpoint=azure_openai_endpoint,
    azure_deployment=azure_openai_deployment  # Nome do deployment do .env (o4-mini não suporta temperature customizada)
)

# <font color="gree">``Passo 5:`` Definimos uma tool customizada para o MathAgent</font>

Agora, criamos uma ferramenta simples que soma dois números. Ela será usada pelo ``Agent 1``. Definimos a função e a decoramos com ``@tool``, para que o ``LangGraph`` saiba como interpretá-la como uma ação chamável.

In [17]:
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

# <font color="gree">``Passo 6:`` Criar os Agentes</font>

Vamos definir dois agentes usando `create_react_agent`:

* ``Agent 1 (Especialista em Matemática):`` Usa nossa ferramenta personalizada de adição e uma tool de transferência para passar o controle para o ``Agent 2``, se necessário.

* ``Agent 2 (Agente de Busca):`` Usa a ``Tavily Search`` e uma toola de transferência de volta para o ``Agent 1``, se necessário.

Também atribuímos a cada agente um prompt e um nome para que eles saibam como se comportar.

In [18]:
from textwrap import dedent

math_agent = create_agent(
    model=model,
    tools=[
        add,
        create_handoff_tool(agent_name="SearchAgent",
                            description=dedent("""Está ferramenta deverá ser usada quando a pergunta do usuário seja
                                                  relacionada a cálculos matemáticos, como soma de dois números.
                                                  Se a pergunta não envolver soma de dois números, transfira para o
                                                  agente: SearchAgent, para pesquisa na internet.
                                               """)
                           )
    ],
    system_prompt=dedent("""
    Você é um agente chamado MathAgent, um assistente útil que é excelente em problemas matemáticos. Se uma pergunta
    precisa de informações externas, transfira para a pergunta do usuário para o agente: SearchAgent.
    Ademais, sempre responda em português do Brasil (pt-br).
    """),
    name="MathAgent",
)

search_agent = create_agent(
    model=model,
    tools=[
        web_search_tool,
        create_handoff_tool(agent_name="MathAgent",
                            description="Transferir para MathAgent para obter ajuda matemática")
    ],
    system_prompt=dedent("""Você é um agente chamado SearchAgent, um assistente útil que usa a internet para
                            responder perguntas do usuário com as seguintes características:
                            * Se for perguntas atuais, use a internet para responder
                            * Se for perguntas relacionadas a história, geografia, ciências, etc, use a internet
                              para responder ao usuário
                            * Se for perguntas externas diferentes de cálculo matemático, use a internet


                            Se o usuário precisar de ajuda matemática, transfira-os para MathAgent.
                            Ademais, sempre responda em português do Brasil (pt-br).
    """),
    name="SearchAgent",
)

# <font color="gree">``Passo 7:`` Configurar memória</font>

Criamos um sistema de memória leve usando o ``InMemorySaver``. Isso permite que os agentes se lembrem da conversa entre etapas e transferências.

In [19]:
checkpointer = InMemorySaver()

# <font color="gree">``Passo 8:`` Construa o Swarm</font>

Combinamos os agentes em um ``Swarm`` usando o comando `create_swarm`. Também informamos ao sistema qual agente deve iniciar a conversa.

In [20]:
workflow = create_swarm(
    agents=[math_agent, search_agent],
    default_active_agent="MathAgent"
)

# <font color="gree">``Passo 9:`` Compilando o App</font>

Compilamos o ``Swarm`` em um aplicativo para que possamos invocá-lo e simular uma conversa. Pense nisso como "ativar" nosso ``sistema multiagente``.

In [21]:
app = workflow.compile(checkpointer=checkpointer)

# <font color="gree">``Passo 10:`` Definimos o ID da thread para esta conversa</font>

Cada conversa precisa de um ``thread_id`` para rastrear o histórico de mensagens. Vamos simplificar e usar ``"1"``.


In [22]:
config = {"configurable": {"thread_id": "1"}}

# <font color="red">Testando nosso Sistema de Agentes Swarm</font>

Enviamos uma mensagem ao enxame com uma consulta que o Agente 2 deve processar, acionando uma transferência do Agente 1 para o agente principal. Além disso, como o Agente 2 possui uma ferramenta que auxilia na resposta à pergunta, essa ferramenta deve ser utilizada.

In [23]:
# Example 1: Pergunta para uma consulta de pesquisa (deve transferir para SearchAgent):
turn_1 = app.invoke(
    {"messages": [{"role": "user",
                   "content": "Quem é o atual presidente do 2026 que está governando o Perú?"}]},
    config
)


turn_1


{'messages': [HumanMessage(content='Quem é o atual presidente do 2026 que está governando o Perú?', additional_kwargs={}, response_metadata={}, id='fe36ae6f-9ce9-4bb3-8a06-f809d315d6ff'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 531, 'prompt_tokens': 184, 'total_tokens': 715, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None, 'id': 'chatcmpl-D2rDJE23tOvobvTt6RKxPpv24JlAh', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filte

In [24]:
resposta = turn_1['messages'][-1].content

resposta

'Em cumprimento ao artigo 115 da Constituição peruana, após a destituição de Dina Boluarte em 10 de outubro de 2025, quem assumiu a Presidência da República foi o então presidente do Congresso, José Enrique Jerí Oré. Ele está exercendo o cargo até o fim do mandato, em 28 de julho de 2026.'

Vamos usar a biblioteca ``Rich``, que ajuda a deixar a saída do terminal mais bonita e colorida.

``from rich import print`` Isso substitui a função `print()` padrão pela versão do Rich, que pode exibir texto formatado, cores, emojis e muito mais no terminal.

``from rich.markdown import Markdown`` Isso permite usar Markdown (uma maneira simples de formatar texto) e exibi-lo de forma organizada no terminal. Por exemplo, você pode mostrar texto em negrito, cabeçalhos, marcadores, etc.

In [25]:
from rich import print
from rich.markdown import Markdown

# Extract agent's response:
first_response = turn_1["messages"][-1]
content = first_response.content

In [26]:
text = content.strip()
markdown = Markdown(text)

# Print with rich formatting
print("[bold green] 🌤️Agent's Response:[/bold green]\n")
print(markdown)

 🌤️Agent's Response:

Em cumprimento ao artigo 115 da Constituição peruana, após a destituição de Dina Boluarte em 10 de outubro de 2025,
quem assumiu a Presidência da República foi o então presidente do Congresso, José Enrique Jerí Oré. Ele está       
exercendo o cargo até o fim do mandato, em 28 de julho de 2026.

Agora, fazemos uma pergunta que o ``Agent 1``, nosso ``MathAgent``, deve responder:

In [27]:
# Example 2: Pergunta para uma pergunta de matemática (deve ficar com MathAgent):
turn_2 = app.invoke(
    {"messages": [{"role": "user",
                   "content": "Quanto é 42 + 58?"}]},
    config
)


In [28]:
second_response = turn_2["messages"][-1]

print("Agent Math:", second_response.content)

Agent Math: O resultado de 42 + 58 é 100.